# Mapear times para regiões e estados

Os dados de universidades e regiões foram gerados quase que manualmente, caso encontre incosistência nos dados pode criar um Issue no repositório.

## Dependências

In [1]:
# encoding: utf-8

import os
import sys

# to import utils
sys.path.append('../')

import pandas as pd
import utils

## Buscar dados como Dataframe Panda

In [3]:
def get_competidores_data(competidores_path='competidores.csv'):
    return pd.read_csv(os.path.join(utils.OUTPUT_PATH, competidores_path))


def get_universidades_data():
    df = pd.read_csv(utils.UNIVERSIDADES_PATH)
    df['sigla'] = [s.upper() for s in df['sigla']]
    return df


def get_regioes_data():
    return pd.read_csv(utils.REGIOES_PATH)

def print_info(df):
    print(df.head(5))

In [4]:
df_competidor = get_competidores_data()
print_info(df_competidor)

    ano                      time                     competidor  reserva  \
0  2016                ufpe - 0xe  lucas almeida pereira de lima        0   
1  2016                ufpe - 0xe                duhan caraciolo        0   
2  2016                ufpe - 0xe        mario henrique da silva        0   
3  2016  unicamp - veni vidi codi        guilherme bueno andrade        0   
4  2016  unicamp - veni vidi codi        matheus de souza ataide        0   

   classificado  posicao medalha  
0             1        1    gold  
1             1        1    gold  
2             1        1    gold  
3             1        2    gold  
4             1        2    gold  


In [5]:
df_universidade = get_universidades_data()
print_info(df_universidade)

                                   universidade     sigla estado
0                       ACADEMIA DA FORÇA AÉREA       AFA     SP
1  ACADEMIA DA POLÍCIA CIVIL DO ESTADO DA BAHIA  ACADEPOL     BA
2   ACADEMIA DE POLÍCIA MILITAR DO BARRO BRANCO     APMBB     SP
3       ACADEMIA DE POLÍCIA MILITAR DOM JOÃO VI   APMDJVI     RJ
4                   ACADEMIA DE POLÍCIA MILITAR       APM     BA


In [6]:
df_regioes = get_regioes_data()
print_info(df_regioes)

     estado sigla    regiao
0      Acre    AC     Norte
1   Alagoas    AL     Norte
2     Amapá    AP     Norte
3  Amazonas    AM     Norte
4     Bahia    BA  Nordeste


## Limpar e criar novas colunas na tabela competidores

In [9]:
def clean_universidade(df):
    '''There are a coulpe of inconsistencies in the data.

         1. DCC-UFRJ -> UFRJ
         2. USP-SãO-PAULO -> USP-SP
         3. USP-SãO-CARLOS -> USP-SC
         4. DCC-UFRJ-NOP -> UFRJ

    '''
    inconsistencies  = {
        'DCC-UFRJ': 'UFRJ',
        'USP-SÃO-PAULO': 'USP-SP',
        'USP-SÃO-CARLOS': 'USP-SC',
        'DCC-UFRJ-NOP': 'UFRJ'
    }
    for k in inconsistencies:
        df.loc[df.universidade.isin([k]), 'universidade'] = inconsistencies[k]

    return df


def clean_time(df):
    '''There are a coulpe of inconsistencies in the data.

         1. dcc ufrj nop -> nop
    '''
    inconsistencies  = {
        'dcc ufrj nop': 'nop'
    }
    for k in inconsistencies:
        df.loc[df.time.isin([k]), 'time'] = inconsistencies[k]
    return df


def add_universidade_column(df):
    # this deals with some different cases like --
    df['time'] = [s.replace('--', '-') for s in df['time']]
    df['universidade'] = [s.split(' - ')[0].replace(' ', '-').upper() for s in df['time']]
    return clean_universidade(df)


def add_time_column(df):
    df['time'] = [s.split(' - ')[-1] for s in df['time']]
    return clean_time(df)


def add_estado_column(df_competidor, df_universidade):
    df_competidor['estado'] = [df_universidade.loc[df_universidade['sigla'] == e]['estado'].iloc[0]
                               for e in df_competidor['universidade']]
    return df_competidor


def add_regioes_column(df_competidor, df_regioes):
    df_competidor['regiao'] = [df_regioes.loc[df_regioes['sigla'] == e]['regiao'].iloc[0]
                               for e in df_competidor['estado']]
    return df_competidor

In [10]:
df_competidor = add_universidade_column(df_competidor)
df_competidor = add_time_column(df_competidor)
df_competidor = add_estado_column(df_competidor, df_universidade)
df_competidor = add_regioes_column(df_competidor, df_regioes)

## Salvar tabela preprocessada como CSV

In [11]:
utils.save_as_csv(df_competidor, os.path.join(utils.OUTPUT_PATH, 'pre_processado_competidores.csv'))